In [3]:
import pandas as pd

skmobhomelocs = pd.read_csv(r'C:\Users\maria\OneDrive\Documents\RESEARCH\Data files\participant home addresses (skmob).csv')

homelocs = pd.read_pickle(r'C:\Users\maria\OneDrive\Documents\RESEARCH\Data files\DRIVES\uid_HomeLocations.pkl')
homelocs = homelocs.reset_index(drop = True) ## dropping arbitrary index present in file
homelocs = homelocs.drop_duplicates()        ## dropping identical rows for each participant, since this file was obtained from another dataset
homelocs[''] = ''                            ## creating an empty column for the addresses

homelocs.columns = ['uid', 'lng', 'lat', 'address'] ## renames columns to match format in homelocationfinder (v1)
homelocs =  homelocs[['uid', 'lat', 'lng', 'address']] ## swaps latitude and longitude columns so lat is first

In [5]:
from geopy.geocoders import GoogleV3
import numpy as np

geolocator = GoogleV3(api_key="xxxx")

for i in range(len(homelocs)):
    try:
        lat = homelocs.iloc[i, 1]
        long = homelocs.iloc[i, 2]
        homelocs.iloc[i, 3] = str(geolocator.reverse(f"{lat}, {long}"))

    except ValueError: ## if participant's home lat/long = NaN
        uid = homelocs.iloc[i, 0] # numpy float
        row = skmobhomelocs[skmobhomelocs['uid'] == str(uid)] ## finds participant's row in skmob home locations file
        homelocs.iloc[i, 1] = row.iloc[0, 1]  ## and inserts skmob location information for that participant
        homelocs.iloc[i, 2] = row.iloc[0, 2]
        homelocs.iloc[i, 3] = row.iloc[0, 3]                

In [7]:
homelocs.to_csv(r'C:\Users\maria\OneDrive\Documents\RESEARCH\Data files\participant home addresses.csv', index = False)